# Bike rentals

We are going to make a linear model to predict the duration of a bike rental. We are going to allow ourselves the pickup and dropoff location of the rental. 

You could change this project to look at the predictions just based on the pickup location if you wanted - which is more suitable depends on whether the cyclist has to give a particular drop-off location, or whether she is allowed to drop the bike off at any station.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

We are able to get information about the San Francisco Bikes from [Kaggle](https://www.kaggle.com/benhamner/sf-bay-area-bike-share). This makes our job quite a bit easier, as we don't have to manually join match the station ids together.

We are going to start by joinging the data for the three features we have (trips, stations, and weather). We are not going to look at the distributions, or do serious EDA yet, as we haven't done a test-train split.

## Loading, joining, cleaning

In [2]:
bike = pd.read_csv('sf_bike/trip.csv.zip')
weather = pd.read_csv('sf_bike/weather.csv')
station = pd.read_csv('sf_bike/station.csv')
bike.head()

,id,duration,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_id,subscription_type,zip_code
0,4576,63,8/29/2013 14:13,South Van Ness at Market,66,8/29/2013 14:14,South Van Ness at Market,66,520,Subscriber,94127
1,4607,70,8/29/2013 14:42,San Jose City Hall,10,8/29/2013 14:43,San Jose City Hall,10,661,Subscriber,95138
2,4130,71,8/29/2013 10:16,Mountain View City Hall,27,8/29/2013 10:17,Mountain View City Hall,27,48,Subscriber,97214
3,4251,77,8/29/2013 11:29,San Jose City Hall,10,8/29/2013 11:30,San Jose City Hall,10,26,Subscriber,95060
4,4299,83,8/29/2013 12:02,South Van Ness at Market,66,8/29/2013 12:04,Market at 10th,67,319,Subscriber,94103


In [3]:
bike.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 669959 entries, 0 to 669958
Data columns (total 11 columns):
id                    669959 non-null int64
duration              669959 non-null int64
start_date            669959 non-null object
start_station_name    669959 non-null object
start_station_id      669959 non-null int64
end_date              669959 non-null object
end_station_name      669959 non-null object
end_station_id        669959 non-null int64
bike_id               669959 non-null int64
subscription_type     669959 non-null object
zip_code              663340 non-null object
dtypes: int64(5), object(6)
memory usage: 56.2+ MB


OK, this is a relatively clean dataset. Some observations:
- The station names appear redundant, we probably just need the station ids. We _might_ be able to use the station ids to look up the locations (and get distances) if we need to.
- `bike_id` and `id` are probably useless
- We can use `start_date`, but not `end_date` if we are trying to find the duration of trips. Note that `start_date` is an object, we should convert it into a datetime.
- `subscription_type` is categorical and will need to be dummified.
- some records (~6000, or 1%) are missing zip codes



In [4]:
bike['start_date'] = pd.to_datetime(bike['start_date'])

In [9]:
bike['is_subscriber'] = pd.get_dummies(bike['subscription_type'], drop_first=True)

In [12]:
# Now drop redundant columns:
cols_to_drop = ['id', 'start_station_name', 'end_station_name', 'end_date', 'bike_id', 'subscription_type']
bike.drop(cols_to_drop, axis=1, inplace=True)
bike.head()

,duration,start_date,start_station_id,end_station_id,zip_code,is_subscriber
0,63,2013-08-29 14:13:00,66,66,94127,1
1,70,2013-08-29 14:42:00,10,10,95138,1
2,71,2013-08-29 10:16:00,27,27,97214,1
3,77,2013-08-29 11:29:00,10,10,95060,1
4,83,2013-08-29 12:02:00,66,67,94103,1


Let's look at the station data frame:

### Station dataframe

In [13]:
station.head()

,id,name,lat,long,dock_count,city,installation_date
0,2,San Jose Diridon Caltrain Station,37.329732,-121.901782,27,San Jose,8/6/2013
1,3,San Jose Civic Center,37.330698,-121.888979,15,San Jose,8/5/2013
2,4,Santa Clara at Almaden,37.333988,-121.894902,11,San Jose,8/6/2013
3,5,Adobe on Almaden,37.331415,-121.893200,19,San Jose,8/5/2013
4,6,San Pedro Square,37.336721,-121.894074,15,San Jose,8/7/2013


Ok, can refine plan:
1. Look at just SF stations
2. Can drop station names in SF data (recover by joining on the id if necessary)
3. Maybe include lat/long. Can use additional data source to get elevation (might be important for biking)


In [14]:
station.city.value_counts()

San Francisco    35
San Jose         16
Mountain View     7
Redwood City      7
Palo Alto         5
Name: city, dtype: int64

In [15]:
station = station[station.city == 'San Francisco']

In [32]:
# update the bike frame
bike_sf = bike[bike['start_station_id'].isin(station.id) & bike['end_station_id'].isin(station.id)]
bike_sf.head()

,duration,start_date,start_station_id,end_station_id,zip_code,is_subscriber
0,63,2013-08-29 14:13:00,66,66,94127,1
4,83,2013-08-29 12:02:00,66,67,94103,1
5,103,2013-08-29 18:54:00,59,59,94109,1
8,113,2013-08-29 17:01:00,66,66,94103,1
10,125,2013-08-29 13:52:00,49,54,94109,1


In [33]:
bike_sf.shape

(603693, 6)

We wanted to add elevation data. We can get elevation from longitude / latitude via [OpenElevation API](https://github.com/Jorl17/open-elevation/blob/master/docs/api.md). The basic format is

```
curl https://api.open-elevation.com/api/v1/lookup\?locations\=10,10\|20,20\|41.161758,-8.583933
```

We will use the `requests` package to do the same thing, passing the parameters via a dictionary (so we don't have to do the messy string escaping ourselves)

In [26]:
import requests

api_url = 'https://api.open-elevation.com/api/v1/lookup'
params = {'locations': '|'.join([f'{s.lat},{s.long}' for _,s in station.iterrows()])}
params

{'locations': '37.795001,-122.39997|37.79728,-122.398436|37.794231,-122.402923|37.795425,-122.404767|37.788975,-122.403452|37.799953,-122.398525|37.790302,-122.390637|37.795392,-122.394203|37.79146400000001,-122.391034|37.78387100000001,-122.408433|37.787152,-122.388013|37.789756,-122.394643|37.792251,-122.39708600000002|37.781752,-122.405127|37.77865,-122.418235|37.781332,-122.418603|37.80477,-122.403234|37.780526,-122.39028799999998|37.785299,-122.39623600000002|37.78697800000001,-122.398108|37.782259,-122.392738|37.771058,-122.402717|37.774814,-122.418954|37.776619,-122.417385|37.784878000000006,-122.401014|37.7766,-122.39547|37.776617,-122.39526|37.788446,-122.408499|37.781039,-122.411748|37.798522,-122.407245|37.794139,-122.394434|37.7913,-122.399051|37.786305,-122.404966|37.789625,-122.400811|37.798541,-122.40086200000002'}

In [27]:
response = requests.get(api_url, params)
response.json()

{'results': [{'elevation': 25, 'latitude': 37.795001, 'longitude': -122.39997},
  {'elevation': 15, 'latitude': 37.79728, 'longitude': -122.398436},
  {'elevation': 46, 'latitude': 37.794231, 'longitude': -122.402923},
  {'elevation': 27, 'latitude': 37.795425, 'longitude': -122.404767},
  {'elevation': 42, 'latitude': 37.788975, 'longitude': -122.403452},
  {'elevation': 9, 'latitude': 37.799953, 'longitude': -122.398525},
  {'elevation': 13, 'latitude': 37.790302, 'longitude': -122.390637},
  {'elevation': 9, 'latitude': 37.795392, 'longitude': -122.394203},
  {'elevation': 13, 'latitude': 37.79146400000001, 'longitude': -122.391034},
  {'elevation': 27, 'latitude': 37.78387100000001, 'longitude': -122.408433},
  {'elevation': 5, 'latitude': 37.787152, 'longitude': -122.388013},
  {'elevation': 28, 'latitude': 37.789756, 'longitude': -122.394643},
  {'elevation': 31, 'latitude': 37.792251, 'longitude': -122.39708600000002},
  {'elevation': 18, 'latitude': 37.781752, 'longitude': -122

In [34]:
# Check the elevation frame works
elevation_frame = pd.DataFrame(response.json()['results'])
elevation_frame.head()


,elevation,latitude,longitude
0,25,37.795001,-122.399970
1,15,37.797280,-122.398436
2,46,37.794231,-122.402923
3,27,37.795425,-122.404767
4,42,37.788975,-122.403452


In [35]:
# Match with the station data
station_with_elevation = pd.merge(station, elevation_frame, left_on=['lat', 'long'], 
                                  right_on=['latitude', 'longitude'], how='left')
station_with_elevation.head()

,id,name,lat,long,dock_count,city,installation_date,elevation,latitude,longitude
0,41,Clay at Battery,37.795001,-122.399970,15,San Francisco,8/19/2013,25,37.795001,-122.399970
1,42,Davis at Jackson,37.797280,-122.398436,15,San Francisco,8/19/2013,15,37.797280,-122.398436
2,45,Commercial at Montgomery,37.794231,-122.402923,15,San Francisco,8/19/2013,46,37.794231,-122.402923
3,46,Washington at Kearney,37.795425,-122.404767,15,San Francisco,8/19/2013,27,37.795425,-122.404767
4,47,Post at Kearney,37.788975,-122.403452,19,San Francisco,8/19/2013,42,37.788975,-122.403452


In [36]:
# note that we have a lot of redundant information
to_drop = ['lat', 'long', 'dock_count', 'city', 'installation_date']
station_with_elevation = station_with_elevation.drop(to_drop, axis=1)
station_with_elevation.head()

,id,name,elevation,latitude,longitude
0,41,Clay at Battery,25,37.795001,-122.399970
1,42,Davis at Jackson,15,37.797280,-122.398436
2,45,Commercial at Montgomery,46,37.794231,-122.402923
3,46,Washington at Kearney,27,37.795425,-122.404767
4,47,Post at Kearney,42,37.788975,-122.403452


### Weather data

In [37]:
## What's in the weather data?
weather.head()

,date,max_temperature_f,mean_temperature_f,min_temperature_f,max_dew_point_f,mean_dew_point_f,min_dew_point_f,max_humidity,mean_humidity,min_humidity,...,mean_visibility_miles,min_visibility_miles,max_wind_Speed_mph,mean_wind_speed_mph,max_gust_speed_mph,precipitation_inches,cloud_cover,events,wind_dir_degrees,zip_code
0,8/29/2013,74.0,68.0,61.0,61.0,58.0,56.0,93.0,75.0,57.0,...,10.0,10.0,23.0,11.0,28.0,0,4.0,NaN,286.0,94107
1,8/30/2013,78.0,69.0,60.0,61.0,58.0,56.0,90.0,70.0,50.0,...,10.0,7.0,29.0,13.0,35.0,0,2.0,NaN,291.0,94107
2,8/31/2013,71.0,64.0,57.0,57.0,56.0,54.0,93.0,75.0,57.0,...,10.0,10.0,26.0,15.0,31.0,0,4.0,NaN,284.0,94107
3,9/1/2013,74.0,66.0,58.0,60.0,56.0,53.0,87.0,68.0,49.0,...,10.0,10.0,25.0,13.0,29.0,0,4.0,NaN,284.0,94107
4,9/2/2013,75.0,69.0,62.0,61.0,60.0,58.0,93.0,77.0,61.0,...,10.0,6.0,23.0,12.0,30.0,0,6.0,NaN,277.0,94107


Notice that the weather data seems to be daily (rather than hourly), and the location gets as fine as zip code. Let's look a little closer at the data:

In [38]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3665 entries, 0 to 3664
Data columns (total 24 columns):
date                              3665 non-null object
max_temperature_f                 3661 non-null float64
mean_temperature_f                3661 non-null float64
min_temperature_f                 3661 non-null float64
max_dew_point_f                   3611 non-null float64
mean_dew_point_f                  3611 non-null float64
min_dew_point_f                   3611 non-null float64
max_humidity                      3611 non-null float64
mean_humidity                     3611 non-null float64
min_humidity                      3611 non-null float64
max_sea_level_pressure_inches     3664 non-null float64
mean_sea_level_pressure_inches    3664 non-null float64
min_sea_level_pressure_inches     3664 non-null float64
max_visibility_miles              3652 non-null float64
mean_visibility_miles             3652 non-null float64
min_visibility_miles              3652 non-null float6

We see that there is some missing data here. We might also guess that a lot of features are correlated (e.g. max and mean temperatures). We will join all the weather data for right now.

In [48]:
# Why is precipitation an object?
# Turns out there is a 'T' for trace amount
weather.precipitation_inches.unique()

array(['0', '0.23', 'T', '0.01', '0.28', '0.63', '0.29', '0.06', '0.85',
       '0.09', '0.64', '0.42', '0.35', '0.43', '0.22', '0.74', '0.03',
       '0.12', '0.16', '0.49', '0.17', '0.08', '0.04', '0.53', '0.07',
       '0.02', '0.83', '1.06', '1.71', '0.37', '0.27', '0.45', '0.78',
       '0.88', '0.66', '0.47', '0.1', '0.61', '0.14', '0.05', '0.68',
       '0.97', '0.26', '0.15', '0.87', '0.57', '0.69', '0.32', '0.21',
       '0.24', '0.52', '0.36', '0.33', '0.25', '0.11', '0.2', '1.18',
       '1.43', '3.12', '0.48', '0.19', '1.09', '0.65', '0.13', '0.91',
       '0.99', '0.18', '0.4', '1.07', nan, '0.41', '0.34', '1.25', '1.85',
       '3.36', '0.71', '1.3', '0.72', '0.6', '0.51', '1.2', '1.28', '3.23',
       '0.55', '1.26', '0.39'], dtype=object)

In [51]:
weather['precipitation_inches'] = weather.precipitation_inches.replace('T', '0.005').astype(float)

In [52]:
weather.events.unique()

array([nan, 'Fog', 'Rain', 'Fog-Rain', 'rain', 'Rain-Thunderstorm'], dtype=object)

In [53]:
# Can probably drop events -- will have large overlap with precipitation_inches
weather.drop(['events'],axis=1,inplace=True)

# Finally convert the date
weather['date'] = pd.to_datetime(weather['date'])

## Joining

(Index(['date', 'max_temperature_f', 'mean_temperature_f', 'min_temperature_f',
        'max_dew_point_f', 'mean_dew_point_f', 'min_dew_point_f',
        'max_humidity', 'mean_humidity', 'min_humidity',
        'max_sea_level_pressure_inches', 'mean_sea_level_pressure_inches',
        'min_sea_level_pressure_inches', 'max_visibility_miles',
        'mean_visibility_miles', 'min_visibility_miles', 'max_wind_Speed_mph',
        'mean_wind_speed_mph', 'max_gust_speed_mph', 'precipitation_inches',
        'cloud_cover', 'wind_dir_degrees', 'zip_code'],
       dtype='object'),
 Index(['id', 'name', 'elevation', 'latitude', 'longitude'], dtype='object'))

In [84]:
def convert_to_float(x):
    try:
        return float(x)
    except ValueError:
        print(x)
        return np.nan
    
# This gives us a (redundant) column to join to the weather
bike_sf['date'] = bike_sf.start_date.dt.date
bike_sf['zip_code']=bike_sf.zip_code.apply(convert_to_float)

/Users/damien/miniconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


94103-2585
94103-2585
94103-2585
94103-2585
94103-2585
94103-2585
94103-2585
94107-3471
94103-2585
94103-2585
94103-2585
94103-2585
94103-2585
94103-2585
94103-2585
94103-2585
94103-2585
94103-2585
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
94103-2585
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
94103-2585
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil
nil


/Users/damien/miniconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [79]:
weather.zip_code.unique()

array([94107, 94063, 94301, 94041, 95113])

In [81]:
bike_sf_weather.zip_code.unique()

array(['94127', '94103', '94109', ..., '2400105', '1004004', '29609'], dtype=object)

In [70]:
bike_sf_weather = pd.merge(bike_sf, weather, how='left', left_on=['zip_code', 'date'],
                           right_on=['zip_code', 'date'])
# now drop the date, as we have the weather joined and start_date includes time
bike_sf_weather.drop(['date'], axis=1, inplace=True)
bike_sf_weather.head()

,duration,start_date,start_station_id,end_station_id,zip_code,is_subscriber,max_temperature_f,mean_temperature_f,min_temperature_f,max_dew_point_f,...,min_sea_level_pressure_inches,max_visibility_miles,mean_visibility_miles,min_visibility_miles,max_wind_Speed_mph,mean_wind_speed_mph,max_gust_speed_mph,precipitation_inches,cloud_cover,wind_dir_degrees
0,63,2013-08-29 14:13:00,66,66,94127,1,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,83,2013-08-29 12:02:00,66,67,94103,1,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,103,2013-08-29 18:54:00,59,59,94109,1,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,113,2013-08-29 17:01:00,66,66,94103,1,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,125,2013-08-29 13:52:00,49,54,94109,1,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [71]:
bike_sf_weather.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 603693 entries, 0 to 603692
Data columns (total 27 columns):
duration                          603693 non-null int64
start_date                        603693 non-null datetime64[ns]
start_station_id                  603693 non-null int64
end_station_id                    603693 non-null int64
zip_code                          597587 non-null object
is_subscriber                     603693 non-null uint8
max_temperature_f                 0 non-null float64
mean_temperature_f                0 non-null float64
min_temperature_f                 0 non-null float64
max_dew_point_f                   0 non-null float64
mean_dew_point_f                  0 non-null float64
min_dew_point_f                   0 non-null float64
max_humidity                      0 non-null float64
mean_humidity                     0 non-null float64
min_humidity                      0 non-null float64
max_sea_level_pressure_inches     0 non-null float64
mean_sea_leve